In [20]:
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json
from pathlib import Path
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.applications import vgg16

In [21]:
facec = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [22]:
#Load JSON file that contains model structure
f = Path("model_structure.json")
model_structure = f.read_text()

In [23]:
#Model building
model = model_from_json(model_structure)

In [24]:
#Weights assignment
model.load_weights("model_weights.h5")

In [25]:
font = cv2.FONT_HERSHEY_SIMPLEX

In [26]:
class VideoCamera(object):
    def __init__(self):
        self.video = cv2.VideoCapture(0)
    def __del__(self):
        self.video.release()
    def get_frame(self):
        _, fr = self.video.read()
        gray_fr = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
        faces = facec.detectMultiScale(gray_fr, 1.3, 5)

        for (x, y, w, h) in faces:
            fc = gray_fr[y:y+h, x:x+w]

            roi = cv2.resize(fc, (48, 48))
            pred = model.predict_emotion(roi[np.newaxis, :, :, np.newaxis])

            cv2.putText(fr, pred, (x, y), font, 1, (255, 255, 0), 2)
            cv2.rectangle(fr,(x,y),(x+w,y+h),(255,0,0),2)

        _, jpeg = cv2.imencode('.jpg', fr)
        return jpeg.tobytes()